In [ ]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import torch
from diffusers import StableDiffusion3Pipeline

# import torch._dynamo
# torch._dynamo.config.suppress_errors = True
torch.backends.cuda.matmul.allow_tf32 = True
# torch.set_float32_matmul_precision("high")
# torch._inductor.config.conv_1x1_as_mm = True
# torch._inductor.config.coordinate_descent_tuning = True
# torch._inductor.config.epilogue_fusion = False
# torch._inductor.config.coordinate_descent_check_all_directions = True

pipe = StableDiffusion3Pipeline.from_pretrained("./models/sd3.5", torch_dtype=torch.float16).to("cuda")
#pipe.enable_xformers_memory_efficient_attention()

# pipe.set_progress_bar_config(disable=True)

# pipe.transformer.to(memory_format=torch.channels_last)
# pipe.vae.to(memory_format=torch.channels_last)

# pipe.transformer = torch.compile(pipe.transformer, mode="max-autotune", fullgraph=True)
# pipe.vae.decode = torch.compile(pipe.vae.decode, mode="max-autotune", fullgraph=True)

# Warm Up
# prompt = "a photo of a cat holding a sign that says hello world"
# for _ in range(3):
#     _ = pipe(prompt=prompt, generator=torch.manual_seed(1))



Loading pipeline components...:   0%|          | 0/9 [00:00<?, ?it/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
with torch.no_grad():
    images = pipe(
        "A capybara holding a sign that reads Hello World",
        num_inference_steps=40,
        guidance_scale=4.5,
        num_images_per_prompt=1,
    ).images

for idx, image in enumerate(images):
    image.save("capybara_{}.png".format(idx))

  0%|          | 0/40 [00:00<?, ?it/s]

In [ ]:
pipe.set_progress_bar_config(disable=True)
TARGET_IMG_COUNT = 10
excel_path = "matched_pairs_similarity95-tiny.xlsx"

import pandas as pd
df = pd.read_excel(excel_path)
names = df.iloc[:, 0].dropna().tolist()

prompt_template = "A person from {name} is working hard. Please generate a clear image showing the person’s face, gender, age, clothing, and background."

for name in names:
    # 创建目录
    output_dir = f"./imgs/{name}"
    os.makedirs(output_dir, exist_ok=True)
        
    # 计算需要生成的图片数量
    existing_images = [f for f in os.listdir(output_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]
    needed_images = max(0, TARGET_IMG_COUNT - len(existing_images))
        
    print(f"Generating {needed_images} images for {name}...")
        
    # 生成图片
    for i in range(needed_images):
        prompt = prompt_template.format(name=name)
        with torch.no_grad():
            image = pipe(
                prompt,
                num_inference_steps=40,
                guidance_scale=4.5,
                num_images_per_prompt=1,
            ).images[0]
        # 保存图片
        image.save(f"{output_dir}/{name}_{len(existing_images)+i+1}.png")
        print(f"Saved {name}_{len(existing_images)+i+1}.png")

Generating 10 images for Yulin City...
Saved Yulin City_1.png
Saved Yulin City_2.png
Saved Yulin City_3.png
Saved Yulin City_4.png
Saved Yulin City_5.png
Saved Yulin City_6.png
Saved Yulin City_7.png
Saved Yulin City_8.png
Saved Yulin City_9.png
Saved Yulin City_10.png
Generating 10 images for Xinyang City...
Saved Xinyang City_1.png
Saved Xinyang City_2.png
Saved Xinyang City_3.png
Saved Xinyang City_4.png
Saved Xinyang City_5.png
Saved Xinyang City_6.png
Saved Xinyang City_7.png
Saved Xinyang City_8.png
Saved Xinyang City_9.png
Saved Xinyang City_10.png
Generating 10 images for Wuzhou City...
Saved Wuzhou City_1.png
Saved Wuzhou City_2.png
Saved Wuzhou City_3.png
Saved Wuzhou City_4.png
Saved Wuzhou City_5.png
Saved Wuzhou City_6.png
Saved Wuzhou City_7.png
Saved Wuzhou City_8.png
Saved Wuzhou City_9.png
Saved Wuzhou City_10.png
Generating 10 images for Suzhou City...
Saved Suzhou City_1.png
Saved Suzhou City_2.png
Saved Suzhou City_3.png
Saved Suzhou City_4.png
Saved Suzhou City_5.p